In [1]:
import numpy as np
import pandas as pd
import requests
import re
import json
from os import walk
from multiprocessing.pool import ThreadPool

In [2]:
URL = "https://s3.amazonaws.com/irs-form-990/index_2017.json"

response = requests.get(URL)
with open("index_2017.json", 'wb') as f:
    f.write(response.content)

In [3]:
with open("index_2017.json") as f:
    data = json.load(f)
    data = data[list(data.keys())[0]]

In [4]:
df = pd.DataFrame.from_dict(data)
df.tail(5)

,EIN,TaxPeriod,DLN,FormType,URL,OrganizationName,SubmittedOn,ObjectId,LastUpdated
489008,272948627,201512,93492320001487,990EZ,https://s3.amazonaws.com/irs-form-990/20173320...,CONSTELLA FESTIVAL OF MUSIC AND FINE ARTS,2017-12-29,201733209349200148,2018-01-18T23:46:56
489009,200745749,201512,93493319184457,990,https://s3.amazonaws.com/irs-form-990/20170319...,CHRIST CHURCH INTERNATIONAL INC,2017-12-28,201703199349318445,2018-01-18T23:46:56
489010,263520140,201608,93492319074127,990EZ,https://s3.amazonaws.com/irs-form-990/20172319...,REBELS SOCCER CLUB,2017-12-27,201723199349207412,2018-01-18T23:53:03
489011,770572762,201707,93491317015447,990PF,https://s3.amazonaws.com/irs-form-990/20174317...,RISING FARMWORKER DREAM FUND,2017-12-29,201743179349101544,2018-01-18T21:48:10
489012,481114983,201704,93493319101237,990,https://s3.amazonaws.com/irs-form-990/20173319...,MADISON EDUCATIONAL ASSISTANCE FUND,2017-12-28,201733199349310123,2018-01-19T00:05:21


In [5]:
df.shape

(489013, 9)

In [6]:
def download_url(url):
    file_name_start = url.rfind('/') + 1
    file_name = url[file_name_start:]
    output_dir = "/home/meso/git_repo/exploring_990/data_2017/"
    file_name = output_dir + file_name
    
    r = requests.get(url, stream=True)
    if r.status_code == requests.codes.ok:
        with open(file_name, 'wb') as f:
            f.write(r.content)
    return url

urls = []
for i in range(df.shape[0]): 
    urls.append(data[i]['URL'])
    
results = ThreadPool(100).imap_unordered(download_url, urls)

In [7]:
f = []
for (dirpath, dirnames, filenames) in walk('/home/meso/git_repo/exploring_990/data_2017/'):
    f.extend(filenames)
    break

In [8]:
u = list(df['URL'])
len(u)

489013

In [9]:
file_names_list = []

for i in range(len(u)):
    file_loc = u[i].rfind("/") + 1
    file_name = u[i][file_loc:]
    file_names_list.append(file_name)

In [10]:
print(len(file_names_list), str(":"), len(f))

489013 : 488680


In [11]:
diff = list(set(file_names_list) - set(f))
len(diff)

333

In [12]:
file_loc = u[i].rfind("/") + 1
url_pre = u[0][:file_loc]

In [13]:
urls = []
for i in range(len(diff)):
    url = url_pre + diff[i]
    urls.append(url)

In [14]:
for url in urls:
    download_url(url)

In [15]:
f = []
for (dirpath, dirnames, filenames) in walk('/home/meso/git_repo/exploring_990/data_2017/'):
    f.extend(filenames)
    break

In [16]:
len(f)

489013

In [17]:
diff = list(set(file_names_list) - set(f))
len(diff)

0

In [18]:
def list_duplicates(seq):
    seen = set()
    seen_add = seen.add
    return [idx for idx, item in enumerate(seq) if item in seen or seen_add(item)]

In [19]:
print(list_duplicates(file_names_list))

[]


In [20]:
dup_df = df.iloc[list_duplicates(file_names_list)]

In [21]:
dup_ein = pd.DataFrame(dup_df['EIN'])

In [22]:
dup_ein

,EIN


In [23]:
duplicates = pd.DataFrame([])

for dup in range(dup_ein.shape[0]):
    ein = dup_ein.iloc[dup]['EIN']
    duplicates = duplicates.append(df[df['EIN'] == ein])

In [24]:
duplicates

""
